### Script purpose: Ming office title coding

1. General principles:
    - A comprehensive ontological structure of office title includes four parts: `Classification + Administrative Unit (optional) + Function (optional) + Title`
    - Each part corresponds to a table.
    - Separate `coding_value` and `raw_value`.
        - `raw_value`: the string appeared in original book text.
        - `coding_value`: the revised string that can be successfully coded.
    - Replace starting from long string to short.
    - Priority: T first, P last.

2. Notes:
    - `Office title by LENGTH` table merges CBDB Ming office title with UCI table. Duplicates in CBDB table are removed in this table, i.e., this is the clean table we are going to use.

### TODO:
- [×] English words;
- [×] Forward slash.

In [1]:
% matplotlib inline
import sqlite3
import pandas as pd
import networkx as nx
import xlrd
import matplotlib.pyplot as plt
import math
import warnings
from tqdm import tqdm
import re
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

### `c_office_chn` from UCI.

In [2]:
df_uci_office_ming=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSCmhbCk1B-9jjINMhy_VwikM6_Sn7bjdO7b_vaZJkVcYCCYlWVlhYVCFtAs0fPX-UEO62GWxaX1qAS/pub?gid=630627340&single=true&output=tsv',
                                    sep='\t')
df_uci_office_ming=df_uci_office_ming[['c_office_id（Dictionary Ser#)','Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn']].rename(columns={'c_office_id（Dictionary Ser#)':'c_office_id'})
df_uci_office_ming['c_office_chn']=[s.replace('/', '') for s in df_uci_office_ming['c_office_chn']]
df_uci_office_ming.sample(3)

,c_office_id,Institution 1,Institution 2,Institution 3,c_office_chn
2300,2657,地方官署類 Regional and Local Governance,省官門 Provincial Governance,行中書省 Branch Secretariat (Abolished in 1376),右司管勾
1022,70153,中央輔佐官署類 Central Administration Assistance,秘書門 Secretary Offices,典籍實錄修纂官,大禮全書總裁官
3063,71781,文武散階勛爵類 Honorary Civil and Military Titles,勛爵門 Noble Titles,公 Duke,豐國公


In [3]:
df_uci_office_ming.sample(10)

,c_office_id,Institution 1,Institution 2,Institution 3,c_office_chn
903,1702,中央輔佐官署類 Central Administration Assistance,寺監門 Courts and Directorates,太醫院 The Imperial Academy of Medicine,院使
544,71180,中央中樞官署類 The Central Government,六部門 Six Ministries,禮部 The Ministry of Rites,行人司司正
2451,70223,地方軍事與治安機構類 Regional and Local Military Units,(行)都指揮使司門 (Auxiliary) Regional and Local Milit...,都指揮使司 The Regional Military Commission,都司僉書管事
1485,1945,京衛京營與中央軍事官署類 Central and Capital Militaries,大都督府門 The Chief Military Commissions,左軍都督府 The Chief Military Commission of the Left,提控案牘
3811,70843,皇族宮廷類 Imperial Family and Royal Court,宦官門 Eunuch Offices,外差宦官 Eunuchs on Secondment,天壽山守備太監
500,71581,中央中樞官署類 The Central Government,六部門 Six Ministries,禮部 The Ministry of Rites,主客清吏司主事
3705,317,皇族宮廷類 Imperial Family and Royal Court,宦官門 Eunuch Offices,印綬監 The Directorate for Credentials,掌印太監
1341,2089,京衛京營與中央軍事官署類 Central and Capital Militaries,京營門 Military Training Units in the Capital,舊三大營 Early Three Training Divisions in the Cap...,五軍營把總
2392,70763,地方軍事與治安機構類 Regional and Local Military Units,(行)都指揮使司門 (Auxiliary) Regional and Local Milit...,衛指揮使司 Guard Military Commands,前衛指揮同知
163,71154,中央中樞官署類 The Central Government,六部門 Six Ministries,刑部 The Ministry of Justice,浙江清吏司主事


In [4]:
df_uci_office_ming['inst_1_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 1']]
df_uci_office_ming['inst_2_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 2']]
df_uci_office_ming['inst_3_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 3']]
df_uci_office_ming['uci_value']=df_uci_office_ming['inst_1_chn']+df_uci_office_ming['inst_2_chn']+'_'+df_uci_office_ming['inst_3_chn']+'_'+df_uci_office_ming['c_office_chn']
df_uci_office_ming['c_office_id']=pd.to_numeric(df_uci_office_ming['c_office_id'], errors='coerce')
df_uci_office_ming.drop(['inst_1_chn', 'inst_2_chn', 'inst_3_chn', 'Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn'], axis=1, inplace=True)

In [5]:
df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()]

,c_office_id,uci_value
1130,71508.0,中央輔佐官署類秘書門_翰林院_直文淵閣侍講學士
1195,71503.0,中央輔佐官署類考官門_會試官_知貢舉官
1219,72165.0,中央輔佐官署類考官門_鄉試官_順天同考官
1282,NaN,京衛京營與中央軍事官署類京營門_京營_京營總兵官
2314,71504.0,地方官署類省官門_行中書省_理問所知事
2718,71274.0,地方軍事與治安機構類招討經略安撫使門_宣撫司_宣撫司經歷
2821,NaN,文武散階勛爵類勛爵門_伯_平涼伯
2842,NaN,文武散階勛爵類勛爵門_伯_新城伯
2862,NaN,文武散階勛爵類勛爵門_伯_永定伯
2882,NaN,文武散階勛爵類勛爵門_伯_鎮遠伯


In [6]:
df_uci_office_ming['uci_value']=[s.replace('/', '') for s in df_uci_office_ming['uci_value']]
df_uci_office_ming['uci_value']=[s.replace('／', '') for s in df_uci_office_ming['uci_value']]
df_uci_office_ming['uci_value']=[s.replace('、', '') for s in df_uci_office_ming['uci_value']]
df_uci_office_ming['uci_value']=[re.sub(r'[a-zA-Z]', string=s, repl='') for s in df_uci_office_ming['uci_value']]

In [7]:
df_uci_office_ming.drop(df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()].index, inplace=True)
df_uci_office_ming.set_index('c_office_id', inplace=True)
df_uci_office_ming.sample(3)

,uci_value
c_office_id,
71082.0,地方官署類縣官門_縣官_縣丞
70731.0,文武散階勛爵類武勛階門__騎都尉
2557.0,地方官署類京府門_順天府_京縣主簿


### `c_office_chn` from CBDB uncleaned, and merge with UCI.

In [8]:
conn = sqlite3.connect('../../SQL/sqlite_20180302.db')
df_cbdb_office_ming=pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn)[pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn).c_dy==19].set_index('c_office_id')
df_cbdb_office_ming.sample(3)

,tts_sysno,c_dy,c_office_pinyin,c_office_chn,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old
c_office_id,,,,,,,,,,,,,,,,
72148,16884.0,19,fu bu zheng shi,副布政使,None,副布政,[Not Yet Translated],None,NaN,None,None,[Not Yet Translated],None,None,None,0.0
72209,16943.0,19,jin yi wei tang wei cheng xi jian dou zhi hui ...,錦衣衛堂衛承禧見都指揮使同錦衣同知,None,None,None,None,NaN,None,None,None,None,None,None,0.0
72588,17319.0,19,jiang shi lang,將仕郎,None,None,None,None,NaN,None,None,None,None,None,None,0.0


In [9]:
for index in tqdm(df_uci_office_ming.index):
    if index in df_cbdb_office_ming.index:
        df_uci_office_ming.loc[index, 'cbdb_value']=df_cbdb_office_ming.loc[index, 'c_office_chn']
        df_uci_office_ming.loc[index, 'tts_sysno']=df_cbdb_office_ming.loc[index, 'tts_sysno']
        df_uci_office_ming.loc[index, 'c_office_pinyin']=df_cbdb_office_ming.loc[index, 'c_office_pinyin']
        df_uci_office_ming.loc[index, 'c_office_pinyin_alt']=df_cbdb_office_ming.loc[index, 'c_office_pinyin_alt']
        df_uci_office_ming.loc[index, 'c_office_chn_alt']=df_cbdb_office_ming.loc[index, 'c_office_chn_alt']
        df_uci_office_ming.loc[index, 'c_office_trans']=df_cbdb_office_ming.loc[index, 'c_office_trans']
        df_uci_office_ming.loc[index, 'c_office_trans_alt']=df_cbdb_office_ming.loc[index, 'c_office_trans_alt']
        df_uci_office_ming.loc[index, 'c_source']=df_cbdb_office_ming.loc[index, 'c_source']
        df_uci_office_ming.loc[index, 'c_pages']=df_cbdb_office_ming.loc[index, 'c_pages']
        df_uci_office_ming.loc[index, 'c_notes']=df_cbdb_office_ming.loc[index, 'c_notes']
        df_uci_office_ming.loc[index, 'c_category_1']=df_cbdb_office_ming.loc[index, 'c_category_1']
        df_uci_office_ming.loc[index, 'c_category_2']=df_cbdb_office_ming.loc[index, 'c_category_2']
        df_uci_office_ming.loc[index, 'c_category_3']=df_cbdb_office_ming.loc[index, 'c_category_3']
        df_uci_office_ming.loc[index, 'c_category_4']=df_cbdb_office_ming.loc[index, 'c_category_4']
        df_uci_office_ming.loc[index, 'c_office_id_old']=df_cbdb_office_ming.loc[index, 'c_office_id_old']
df_uci_office_ming.loc[index, 'c_dy']=19

100%|██████████| 4304/4304 [01:05<00:00, 66.11it/s]


In [10]:
df_office_ming_merged=df_uci_office_ming
df_office_ming_merged.sample(3)

,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy
c_office_id,,,,,,,,,,,,,,,,,
2758.0,地方官署類州官門_州官_關副使,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
816.0,皇族宮廷類女官門_尚寢局_司設司典設,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2499.0,牧鹽舶政類鹽課鹽運門_都轉鹽運使司_鹽倉大使,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Coding `c_office_chn`.

### TODO: - Done.
    - [×] Subtract titles from right.
    - [×] Add appointment type.
    - [×] Use online revised CLS table.

In [11]:
df_adm=pd.read_csv('../data_output/C_OT_ADM.tsv', sep='\t').set_index('c_ot_adm_id')
df_cls=pd.read_csv('../data_output/C_OT_CLS.tsv', sep='\t').set_index('c_ot_cls_id')
df_tit=pd.read_csv('../data_output/C_OT_TIT.tsv', sep='\t').set_index('c_ot_tit_id')
df_func=pd.read_csv('../data_output/C_OT_FUNC.tsv', sep='\t').set_index('c_ot_func_id')
df_app_ty=pd.read_csv('../data_output/APPOINTMENT_TYPE_CODES.tsv', sep='\t').set_index('c_appt_type_code')
df_txt_code=pd.read_csv('../data_output/TEXT_CODES.tsv', sep='\t').set_index('c_textid')

In [12]:
df_tit.sample(3)

,c_ot_tit_chinm,value_to_run,c_ot_tit_desc,c_ot_tit_start,c_ot_tit_end,length
c_ot_tit_id,,,,,,
742,臨江侯,2.0,NaN,NaN,NaN,3
1745,路總兵官,1.0,NaN,NaN,NaN,4
299,靖遠侯,2.0,NaN,NaN,NaN,3


### Choose either one.

#### Use coding_value.

In [17]:
df_coding_value=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwXjRmlMR9w2ZV2tcenPSz9UgE7WAgeumGxxCJlceQOZRQFgm6_mgMCAlC_GzM0yxxNsDOlU1-5aH-/pub?output=xlsx',
                              sheetname='merged_tbl_coding'
                             )[['c_office_id', 'coding_value']].set_index('c_office_id')

In [18]:
df_coding_value.sample(3)

,coding_value
c_office_id,
70025.0,皇族宮廷類宗室門諸王邠王
1063.0,中央中樞官署類六部門戶部民部員外郎
72095.0,皇族宮廷類宗室門藩王雍王


In [19]:
for c_office_id in tqdm(df_office_ming_merged.index):
    df_office_ming_merged.loc[c_office_id, 'c_ot_coding']=df_coding_value.loc[c_office_id, 'coding_value']

100%|██████████| 4304/4304 [00:04<00:00, 864.00it/s]


#### Use UCI value.

In [ ]:
df_office_ming_merged['c_ot_coding']=df_office_ming_merged['uci_value']

### Begin to replace.

In [ ]:
# Replace titles (only one title in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    ming_ot = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
    ming_ot_done=[]
    for tit_index in df_tit.index:
        tit=df_tit.loc[tit_index, 'c_ot_tit_chinm']
        if ming_ot.endswith(tit) and ming_ot not in ming_ot_done:
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_tit_chinm']=tit
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=re.findall(r'(.+)'+tit, ming_ot)[0]+'T'+str(tit_index)
            ming_ot_done.append(ming_ot)
df_office_ming_merged.sample(3)

 96%|█████████▌| 4140/4304 [04:55<00:11, 14.02it/s]

#### Run on first part.

In [ ]:
# Replace Classifications (can have multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    cls_list=[]
    for cls_index in df_cls.index:
        cls=df_cls.loc[cls_index, 'c_ot_cls_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding'].split('_') # Only use the first part, i.e., classifications.
        if cls in c_ot_coding[0]:
            cls_list.append(cls)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding[0].replace(cls, 'C'+str(cls_index))+'_'+c_ot_coding[1]+'_'+c_ot_coding[2] # Add left parts.
    if cls_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_cls_chinm']='#'.join(cls_list)
df_office_ming_merged.sample(3)

#### Run on second part.

In [ ]:
# Replace admin units (can have multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    adm_list=[]
    for adm_index in df_adm.index:
        adm=df_adm.loc[adm_index, 'c_ot_adm_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding'].split('_')
        if adm in c_ot_coding[1]:
            adm_list.append(adm)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding[0]+'_'+c_ot_coding[1].replace(adm, 'A'+str(adm_index))+'_'+c_ot_coding[2]
    if adm_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_adm_chinm']='#'.join(adm_list)
df_office_ming_merged.sample(3)

In [ ]:
# Run Classifications on second part.
for ming_ot_index in tqdm(df_office_ming_merged.index):
    cls_list=[]
    for cls_index in df_cls.index:
        cls=df_cls.loc[cls_index, 'c_ot_cls_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding'].split('_') # Only use the first part, i.e., classifications.
        if cls in c_ot_coding[1]:
            cls_list.append(cls)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding[0]+'_'+c_ot_coding[1].replace(cls, 'C'+str(cls_index))+'_'+c_ot_coding[2] # Add left parts.
    if cls_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_cls_chinm']='#'.join(cls_list)
df_office_ming_merged.sample(3)

#### Run on third part.

In [ ]:
# Replace admin units (can have multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    adm_list=[]
    for adm_index in df_adm.index:
        adm=df_adm.loc[adm_index, 'c_ot_adm_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding'].split('_')
        if adm in c_ot_coding[2]:
            adm_list.append(adm)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding[0]+'_'+c_ot_coding[1]+'_'+c_ot_coding[2].replace(adm, 'A'+str(adm_index))
    if adm_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_adm_chinm']='#'.join(adm_list)
df_office_ming_merged.sample(3)

In [ ]:
# Replace functional units (can have multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    func_list=[]
    for func_index in df_func.index:
        func=df_func.loc[func_index, 'c_ot_func_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding'].split('_')
        if func in c_ot_coding[2]:
            func_list.append(func)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding[0]+'_'+c_ot_coding[1]+'_'+c_ot_coding[2].replace(func, 'F'+str(func_index))
    if func_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_func_chinm']='#'.join(func_list)
df_office_ming_merged.sample(3)

In [ ]:
# Replace text code.
for ming_ot_index in tqdm(df_office_ming_merged.index):
    txt_list=[]
    for txt_index in df_txt_code.index:
        txt=df_txt_code.loc[txt_index, 'c_title_chn']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if txt in c_ot_coding:
            txt_list.append(txt)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(txt, 'B'+str(txt_index))
    if txt_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_func_chinm']='#'.join(txt_list)
df_office_ming_merged.sample(3)

In [ ]:
# Replace appointment type.
for ming_ot_index in tqdm(df_office_ming_merged.index):
    app_ty_list=[]
    for app_ty_index in df_app_ty.index:
        app_ty=df_app_ty.loc[app_ty_index, 'c_appt_type_desc_chn']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding'].split('_')
        if app_ty in c_ot_coding[2]:
            app_ty_list.append(app_ty)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding[0]+'_'+c_ot_coding[1]+'_'+c_ot_coding[2].replace(app_ty, 'P'+str(app_ty_index))
    if app_ty_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_app_chinm']='#'.join(app_ty_list)
df_office_ming_merged.sample(3)

In [ ]:
for index in tqdm(df_office_ming_merged.index):
    c_ot_coding=df_office_ming_merged.loc[index, 'c_ot_coding']
    if re.sub(r'A|C|T|F|P|（|）|B|\d', '', string=c_ot_coding)!='':
        df_office_ming_merged.loc[index, 'pass']='F'
    else:
        df_office_ming_merged.loc[index, 'pass']='T'

In [ ]:
# Retain the 'type' column online.
df_office_ming_merged_coded=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwXjRmlMR9w2ZV2tcenPSz9UgE7WAgeumGxxCJlceQOZRQFgm6_mgMCAlC_GzM0yxxNsDOlU1-5aH-/pub?output=xlsx',
                                          sheetname='merged_tbl_coding'
                                         )
df_office_ming_merged_coded.set_index('c_office_id', inplace=True)
for c_office_id in df_office_ming_merged.index:
    df_office_ming_merged.loc[c_office_id, 'type']=df_office_ming_merged_coded.loc[c_office_id, 'type']
    df_office_ming_merged.loc[c_office_id, 'c_title_chn']=df_office_ming_merged_coded.loc[c_office_id, 'c_title_chn']
    df_office_ming_merged.loc[c_office_id, 'book_raw']=df_office_ming_merged_coded.loc[c_office_id, 'book_raw']
    df_office_ming_merged.loc[c_office_id, 'c_textid']=df_office_ming_merged_coded.loc[c_office_id, 'c_textid']

In [ ]:
df_office_ming_merged.to_excel('../data_output/ming_office_title_merged_coding.xlsx', encoding='utf8')